Будем решать задачу классификации вина по его химическим свойствам.

In [21]:
import pandas as pd
from sklearn.datasets import load_wine

data = load_wine(as_frame = True)

X = data.data
X = X[X.columns[:7]]

y = data.target

In [22]:
X.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69


Посмотрим на целевую переменную

In [23]:
y.value_counts()

1    71
0    59
2    48
Name: target, dtype: int64

Классы не очень сбалансированы, поэтому для оценки качества классификации будем использовать метрику $f_1$-score.

Разобъем данные на train и test

In [24]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=42)

Приведем признаки к одному масштабу

In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(Xtrain)

Xtrain = pd.DataFrame(scaler.transform(Xtrain), columns=X.columns)
Xtest = pd.DataFrame(scaler.transform(Xtest), columns=X.columns)

Обучим метод опорных векторов на тренировочных данных и оценим качество на тестовых данных

In [26]:
from sklearn.svm import SVC

model = SVC(kernel = 'linear')

model.fit(Xtrain, ytrain)

SVC(kernel='linear')

In [27]:
from sklearn.metrics import f1_score

pred = model.predict(Xtest)

f1_score(ytest, pred, average='weighted')

0.8890522875816994

У SVM как и других линейных моделей есть регуляризация.

Гиперпараметр С регулирует силу регуляризации. Давайте посмотрим, как меняется качество модели при различных С.

In [28]:
import numpy as np
from sklearn.model_selection import cross_val_score

for C in np.arange(0.01, 1.1, 0.1):
    print('C:', C.round(2), 'score=', cross_val_score(SVC(kernel = 'linear', C=C), X, y, cv=3, scoring='f1_weighted').mean().round(3))

C: 0.01 score= 0.801
C: 0.11 score= 0.881
C: 0.21 score= 0.898
C: 0.31 score= 0.904
C: 0.41 score= 0.898


C: 0.51 score= 0.91
C: 0.61 score= 0.916
C: 0.71 score= 0.915
C: 0.81 score= 0.92
C: 0.91 score= 0.915
C: 1.01 score= 0.915


In [29]:
model = SVC(kernel = 'linear', C=0.8)

model.fit(Xtrain, ytrain)

pred = model.predict(Xtest)

f1_score(ytest, pred, average='weighted')

0.912826474463303

In [30]:
model.predict_proba(Xtest)

AttributeError: predict_proba is not available when  probability=False

Также у SVM можно настраивать и другие гиперпараметры:

*  class_weight - при сильном дисбалансе классов можно ставить значение 'balanced'

*  probability - если требуется предсказывать вероятности классов, то нужно ставить значение True

*  decision_function_shape - по умолчанию 'ovr' (для многоклассовой классификации можно пробовать опцию 'ovo')